In [ ]:
#COUPLE OF NECESSARY IMPORTS
import matplotlib.pyplot as plt
%matplotlib inline
import os
import numpy as np
from skimage import io

In [ ]:
datadir = "/home/nitinbindal/Downloads/dataset/Training"  #Training folder path to your dataset
categories = ["Apple Golden 1","Apple Golden 2","Apple Golden 3"]
img_array=[]


In [ ]:
#Converting the images into an array to perform CNNs
for category in categories:
    print(categories.index(category))
    path = os.path.join(datadir,category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        img_array.append([io.imread(os.path.join(path,img)),class_num])

In [ ]:
import random 
random.shuffle(img_array)

In [ ]:
data = np.array(img_array)

In [ ]:
datadir2 = "/home/nitinbindal/Downloads/dataset/Test"   #Test folder path of your dataset
test_arr =[]

In [ ]:
for category in categories:
    path = os.path.join(datadir2,category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        test_arr.append([io.imread(os.path.join(path,img)),class_num])

In [ ]:
def one_hot_encode(vec,vals=3):     #Performing one hot encoding
    n = len(vec)
    out = np.zeros((n,vals))
    out[range(n),vec]=1
    return out

In [ ]:
training_images =[]
training_labels =[]
test_images =[]
test_labels =[]
batch_size =50

In [ ]:
for features,labels in data:
    training_images.append(features)
    training_labels.append(labels)

In [ ]:
for feature,label in test_arr:
    test_images.append(feature)
    test_labels.append(label)
    

In [ ]:
test_image = np.array(test_images)/255     # --NORMALIZATION OF RGB
test_label = one_hot_encode(np.hstack(np.array(test_labels)))

In [ ]:
dat=np.array(training_images)/255
dat[0:50].shape

In [ ]:
class next_func:
    def __init__(self):
        self.i = 0
        self.training_images = np.array(training_images)/255      # --NORMALIZATION OF RGB
        self.training_labels = one_hot_encode(np.hstack(np.array(training_labels)))
    def next_batch(self,batch_size):
        X = self.training_images[self.i:self.i + batch_size]
        y = self.training_labels[self.i:self.i+batch_size]
        self.i = (self.i + batch_size) % len(self.training_images)
        return X, y

In [ ]:
fb = next_func() 

In [ ]:
import tensorflow as tf

In [ ]:
X = tf.placeholder(tf.float32,[None,100,100,3])    #None is batch-size
y = tf.placeholder(tf.float32,[None,3])

In [ ]:
hold_prob = tf.placeholder(tf.float32)

In [ ]:
#HELPER FUNCTIONS

In [ ]:
def init_weights(shape):
    rand_list = tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(rand_list)

In [ ]:
def init_bias(shape):
    rand_list = tf.constant(0.1,shape=shape)
    return tf.Variable(rand_list)

In [ ]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")

In [ ]:
def avg_pool(X):
    return tf.nn.avg_pool(X,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [ ]:
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [ ]:
he_init = tf.contrib.layers.variance_scaling_initializer()

In [ ]:
def normal_layer(input_x,neurons,actf=None):
    return tf.layers.dense(input_x,neurons,activation = actf,kernel_initializer=he_init)

In [ ]:
convo_1 = convolutional_layer(X,[5,5,3,32])
pool1 = avg_pool(convo_1)

In [ ]:
convo_2 = convolutional_layer(pool1,[5,5,32,64])
pool2 = avg_pool(convo_2)

In [ ]:
convo_3 = convolutional_layer(pool2,[7,7,64,128])

In [ ]:
convo_3_flat = tf.reshape(convo_3,[-1,25*25*128])   #On 100 by 100 pixels you have applied 2 times avg_pooling means half the size each time hence (100/2)/2=25

In [ ]:
fully_connected = normal_layer(convo_3_flat,2048,actf = tf.nn.elu)  #USING LEAKY RELU FUNCTION

In [ ]:
dropout = tf.nn.dropout(fully_connected,keep_prob = hold_prob)

In [ ]:
y_pred = normal_layer(dropout,3,actf=None)

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred,labels=y))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate =0.0001)
train_op = optimizer.minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.98)  #USE THIS OPTION ONLY WHEN YOUR TENSORFLOW IS INSTALLED ON GPU

In [ ]:
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    init.run()
    
    for i in range(1200):
        X_batch,y_batch = fb.next_batch(batch_size)
        sess.run(train_op,feed_dict={X:X_batch,y:y_batch,hold_prob:0.5})
        print(i)
        if i%100 ==0:
            print("STEP :{}".format(i))
            correct = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
            print(accuracy.eval(feed_dict={X:test_image,y:test_label,hold_prob:1.0}))
            print("\n")
        
     #Highest accepted accuracy is 99.7955% if you can increase this let me know that!!!   